In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 **Importing all Libraries**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('../input/fer2013/fer2013.csv')

In [ ]:
# INTERESTED_LABELS = [0, 2, 3, 4, 5, 6]
# data = data[data.emotion.isin(INTERESTED_LABELS)]
data

In [ ]:
x_data = data['pixels']
y_data = data['emotion']

**emotion_labels = ['0:angry','1:disgust','2:fear','3:Happy', '4:Sad','5:Surprise', '7:Neutral']**

**DATA REPRESENTING ON BAR GRAPH**

In [ ]:
#data ploting on bar graph 
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="emotion", data=data)

**BALINCING THE DATA USING RANDOM OVER SAMPLING**

In [ ]:
#sampling strategy set to auto, smapling done based on no of observations having majority class 
oversampler = RandomOverSampler(sampling_strategy='auto')
#reshape(-1,1), reshaping  unknown count of the rows and in 1 column##values
x_data, y_data = oversampler.fit_resample(x_data.values.reshape(-1,1), y_data)
print(x_data.shape," ",y_data.shape)

In [ ]:
# value_count=it returns an object that containing counts of unquie values
y_data.value_counts() 

In [ ]:
#flatten:Return a copy of the array collapsed into one dimension
x_data = pd.Series(x_data.flatten())
x_data

In [ ]:
# mapping string split to x_data to dividing string data without having white space b/w them 
#a=['shanmukh john','nani devena', 'raju pawan'] ==> a=['shanmukh','john','nani','devena','raju','pawan']
#np.float32 converts a number stored in a string or integer into a floating point number and in 32 bits
x_data = np.array(list(map(str.split, x_data)), np.float32)
#normalizing pixel values from 0 to 1
x_data/=255
#printing upto 10 elemets
x_data[:10] 

In [ ]:
#reshape(-1, 48, 48, 1):image size 48*48
x_data = x_data.reshape(-1, 48, 48, 1)
x_data.shape 

In [ ]:
y_data = np.array(y_data)
#shape[0]:gives no of rows 
y_data = y_data.reshape(y_data.shape[0], 1)
y_data.shape

**SPLITING THE DATASET INTO TRAIN AND TEST**

In [ ]:
#test size =10%
#setting a number to random state gives training data is same while running multiple times
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, random_state = 45)


***BUILDING A MODEL***

In [ ]:
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(128, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(128, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.2),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()


In [ ]:
# tf.keras.utils.plot_model(model, to_file = "model.png", show_shapes = True, show_dtype = True)

**MODEL COMPILING**

In [ ]:
#model.Compile defines the loss function, the optimizer and the metrics
model.compile(optimizer = Adam(learning_rate=0.001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])


**CALLBACKS**

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
#to_categorical:Converts a class vector (integers) to binary class matrix
#we are using categorical_crossentopy loss function, labels to be provided in a one_hot representation
#one hot vector:1*N matrix in banary
y_train = np_utils.to_categorical(y_train, 7)
y_train.shape

In [ ]:
y_test = np_utils.to_categorical(y_test, 7)
y_test.shape

***FITTING THE MODEL***

In [ ]:
history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks,)


**EVALUATION**

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
#plotting graph to show model accuracy and modle loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
##emotion_labels = ['0:angry','1:disgust','2:fear','3:Happy', '4:Sad','5:Surprise', '7:Neutral']
y_pred = model.predict(x_test)
y_result = []

for pred in y_pred:
    y_result.append(np.argmax(pred))
y_result[:10]

In [ ]:
y_actual = []

for pred in y_test:
    y_actual.append(np.argmax(pred))
y_actual[:10]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_actual, y_result))

In [ ]:
import seaborn as sn
cm = tf.math.confusion_matrix(labels = y_actual, predictions = y_result)

plt.figure(figsize = (10, 7))
sn.heatmap(cm, annot = True, fmt = 'd')
plt.xlabel('Predicted')
plt.ylabel('Truth')

**Saving The Model**

In [ ]:
# model.save('fer2.h5')

In [ ]:
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.6),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Adam(learning_rate=0.0001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
from tensorflow.keras.optimizers import SGD
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.6),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = SGD(learning_rate=0.0001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 10, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
from tensorflow.keras.optimizers import Nadam
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.6),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Nadam(learning_rate=0.0001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 10, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
# Dropout
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.2),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Adam(learning_rate=0.0001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
# Dropout
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.4),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Adam(learning_rate=0.0001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
# learning rate
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.6),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Adam(learning_rate=0.00001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks)

In [ ]:
# learning rate
model = Sequential([
    Input((48, 48, 1)),
    #spatial dimention
    #1st conv layer with 32 filetrs, and kernal size(matrix_size) is 3*3, filter step to next pixel with stride (1,1), valid padding 
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #2nd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    #axis=3:it represnts dimentions(height,width,channel)
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #3rd conv layer
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    #4th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #5th conv layer
    Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    #flatten(): it converts into 1- dimention array inputting to next layer
    Flatten(),
    #fully connected network with 200 neurons
    Dense(200, activation='relu'),
    #dropout rate is 0.6
    Dropout(0.6),
    #output with neurons
    Dense(7, activation = 'softmax')
])
model.summary()

model.compile(optimizer = Adam(learning_rate=0.000001,decay=1e-6),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history=model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test),callbacks = callbacks)